In [5]:
import numpy as np
import pandas as pd

import matplotlib # Para ver la versión
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("madrid_rental_properties_raw_1.csv")

df

,url,property_native_id,rent_eur_per_month,barrio,distrito,latitude,longitude,listing_type,scrape_status,scraped_timestamp,...,Superficie solar,Superficie útil,Teléfono,Terraza,Tipo de casa,Tipo suelo,Trastero,Urbanizado,Vidrios dobles,page_source
0,https://www.pisos.com/alquilar/atico-salamanca...,5.174066e+10,10000.0,Castellana,Salamanca,"40,4342811","-3,6862864",rental,Success,2025-05-28T19:53:30.586256,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,1
1,https://www.pisos.com/alquilar/atico-rio_rosas...,5.423369e+10,1450.0,Río Rosas,Chamberí,"40,4386119","-3,6999008",rental,Success,2025-05-28T19:53:32.370621,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,1
2,https://www.pisos.com/alquilar/piso-salamanca_...,5.423565e+10,2200.0,Castellana,Salamanca,"40,4352372","-3,6835148",rental,Success,2025-05-28T19:53:34.120400,...,NaN,100 m²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,https://www.pisos.com/alquilar/piso-fuente_del...,5.752243e+08,1700.0,Fuente del Berro,Salamanca,40.4290471,-3.6664739,rental,Success,2025-05-28T19:53:35.912036,...,NaN,50 m²,NaN,True,NaN,Grés,NaN,NaN,NaN,1
4,https://www.pisos.com/alquilar/piso-justicia_c...,5.252461e+10,2650.0,Justicia-Chueca,Centro,40.4226065,-3.6991962,rental,Success,2025-05-28T19:53:37.641884,...,NaN,77 m²,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,https://www.pisos.com/alquilar/chalet_pareado-...,1.036083e+07,4300.0,NaN,NaN,"40,516174012","-3,640805874",rental,Success,2025-05-28T20:58:10.540537,...,NaN,270 m²,NaN,Solarium,NaN,NaN,NaN,NaN,NaN,73
2185,https://www.pisos.com/alquilar/chalet_adosado-...,3.839881e+10,6000.0,NaN,NaN,"40,516125912","-3,640196987",rental,Success,2025-05-28T20:58:12.307336,...,NaN,270 m²,NaN,NaN,NaN,Tarima flotante,NaN,True,True,73
2186,https://www.pisos.com/alquilar/chalet_unifamil...,9.417805e+10,6000.0,NaN,NaN,"40,5211707","-3,6484008",rental,Success,2025-05-28T20:58:14.052258,...,355 m²,350 m²,NaN,NaN,NaN,Mármol,NaN,NaN,True,73
2187,https://www.pisos.com/alquilar/piso-el_soto_de...,5.092987e+10,5500.0,NaN,NaN,"40,5198911","-3,64234",rental,Success,2025-05-28T20:58:15.822177,...,NaN,250 m²,NaN,Orientada al sur,NaN,NaN,True,NaN,True,73


In [6]:
# 1. Limpiar nombres de columnas: eliminar espacios y estandarizar todo en minúsculas
df.columns = (df.columns.str.strip().str.lower())

# 2. Reemplazar comas por puntos en todas las celdas de tipo string
df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

# 3. Función para limpiar y convertir superficies a float
def limpiar_superficie(col):
    return pd.to_numeric(
        df[col].astype(str).str.replace(' m²', '', regex=False).str.replace(',', '.'),
        errors='coerce'
    )

df['superficie construida'] = limpiar_superficie('superficie construida')
df['superficie útil'] = limpiar_superficie('superficie útil')


# 4. Normalizar columna planta
def normalizar_planta(planta):
    if pd.isna(planta):
        return planta
    planta = str(planta).strip().lower()
    if "semisótano" in planta or "entresuelo" in planta or "sótano" in planta:
        return -1
    if "principal" in planta or "bajo" in planta:
        return 0
    if "más de 20" in planta:
        return 20
    planta = planta.replace("ª", "").replace("º", "")
    try:
        return int(planta)
    except ValueError:
        return planta  

df["planta"] = df["planta"].apply(normalizar_planta)

# 5. Intentar convertir todas las columnas a float (si aplica)
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    except:
        pass  # Ignora columnas que no pueden convertirse

# 6. Eliminar filas duplicadas
df.drop_duplicates(inplace=True)

# 7. Eliminar columnas completamente vacías
df.dropna(axis=1, how='all', inplace=True)

# 8. Reiniciar índice ---
df.reset_index(drop=True, inplace=True)

df


C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\2399255407.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)


C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\2399255407.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


,url,property_native_id,rent_eur_per_month,barrio,distrito,latitude,longitude,listing_type,scrape_status,scraped_timestamp,...,superficie solar,superficie útil,teléfono,terraza,tipo de casa,tipo suelo,trastero,urbanizado,vidrios dobles,page_source
0,https://www.pisos.com/alquilar/atico-salamanca...,5.174066e+10,10000.0,Castellana,Salamanca,40.434281,-3.686286,rental,Success,2025-05-28T19:53:30.586256,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,1
1,https://www.pisos.com/alquilar/atico-rio_rosas...,5.423369e+10,1450.0,Río Rosas,Chamberí,40.438612,-3.699901,rental,Success,2025-05-28T19:53:32.370621,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,1
2,https://www.pisos.com/alquilar/piso-salamanca_...,5.423565e+10,2200.0,Castellana,Salamanca,40.435237,-3.683515,rental,Success,2025-05-28T19:53:34.120400,...,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,https://www.pisos.com/alquilar/piso-fuente_del...,5.752243e+08,1700.0,Fuente del Berro,Salamanca,40.429047,-3.666474,rental,Success,2025-05-28T19:53:35.912036,...,NaN,50.0,NaN,True,NaN,Grés,NaN,NaN,NaN,1
4,https://www.pisos.com/alquilar/piso-justicia_c...,5.252461e+10,2650.0,Justicia-Chueca,Centro,40.422607,-3.699196,rental,Success,2025-05-28T19:53:37.641884,...,NaN,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,https://www.pisos.com/alquilar/chalet_pareado-...,1.036083e+07,4300.0,NaN,NaN,40.516174,-3.640806,rental,Success,2025-05-28T20:58:10.540537,...,NaN,270.0,NaN,Solarium,NaN,NaN,NaN,NaN,NaN,73
2185,https://www.pisos.com/alquilar/chalet_adosado-...,3.839881e+10,6000.0,NaN,NaN,40.516126,-3.640197,rental,Success,2025-05-28T20:58:12.307336,...,NaN,270.0,NaN,NaN,NaN,Tarima flotante,NaN,1.0,True,73
2186,https://www.pisos.com/alquilar/chalet_unifamil...,9.417805e+10,6000.0,NaN,NaN,40.521171,-3.648401,rental,Success,2025-05-28T20:58:14.052258,...,355 m²,350.0,NaN,NaN,NaN,Mármol,NaN,NaN,True,73
2187,https://www.pisos.com/alquilar/piso-el_soto_de...,5.092987e+10,5500.0,NaN,NaN,40.519891,-3.642340,rental,Success,2025-05-28T20:58:15.822177,...,NaN,250.0,NaN,Orientada al sur,NaN,NaN,True,NaN,True,73


In [7]:
# Eliminar las columnas no relevantes

columns_to_drop = ['url', 'property_native_id', 'scrape_status', 'scraped_timestamp', 'page_source', 'listing_type'
                   'referencia', 'description', 'energy_certificate_main_classification', 'energy_consumption_value',
                    'energy_emissions_value', 'luz', 'no se aceptan mascotas', 'urbanizado', 'calle asfaltada',
                    'calle alumbrada', 'tipo de casa', 'luz', 'se aceptan mascotas', 'superficie solar', 'teléfono',
                    'carpintería interior', 'interior', 'agua', 'soleado', 'carpintería exterior', 'tipo suelo',
                     'gastos de comunidad',  'gas', 'lavadero', 'comedor', 'orientación', 'armarios empotrados']

df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Eliminar las filas de datos cuyo precio de alquiler es NaN
df = df.dropna(subset=["rent_eur_per_month"])

# Rellenar NaN en superficie útil con el valor de superficie construida
df['superficie útil'] = df['superficie útil'].fillna(df['superficie construida'])

# Función para rellenar baños si es NaN
def rellenar_banos(row):
    if pd.isna(row['baños']):
        if not pd.isna(row['superficie construida']):
            return max(1, round(row['superficie construida'] / 70)) 
        else:
            return 1   # Al menos 1 baño
    else:
        return int(row['baños'])

df['baños'] = df.apply(rellenar_banos, axis=1)

import numpy as np

# Función para imputar habitaciones según superficie útil
def estimar_habitaciones(row):
    if pd.isna(row['habitaciones']):
        if not pd.isna(row['superficie útil']):
            sup = row['superficie útil']
            if sup < 50:
                return 1
            else:
                return int(np.floor(sup / 25))
        else:
            return np.nan  # Si tampoco hay superficie útil, no se puede estimar
    else:
        return row['habitaciones']

df['habitaciones'] = df.apply(estimar_habitaciones, axis=1)


# Sustituir NaN por 0 en las columnas indicadas
columnas_a_llenar_cero = ['adaptado a personas con movilidad reducida', 'planta']

for col in columnas_a_llenar_cero:
    if col in df.columns:
        df[col].fillna(0, inplace=True)


# Sustituir NaN por False en las columnas indicadas 
columnas_a_llenar_false = ['amueblado', 'energy_consumption_rating', 'energy_emissions_rating', 'aire acondicionado',
                           'calefacción', 'cocina equipada', 'garaje', 'exterior', 'trastero', 'ascensor', 'terraza',
                           'balcón', 'chimenea', 'piscina', 'jardín', 'vidrios dobles', 'sistema de seguridad', 
                           'puerta blindada', 'portero automático']

for col in columnas_a_llenar_false:
    if col in df.columns:
        df[col].fillna(False, inplace=True)


df['conservación'] = df['conservación'].fillna('En buen estado')


C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\520059015.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['superficie útil'] = df['superficie útil'].fillna(df['superficie construida'])
C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\520059015.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['baños'] = df.apply(rellenar_banos, axis=1)
C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\520059015.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [8]:
# Columnas con más del 80% de nulos
nulls = df.isnull().mean().sort_values(ascending=False)
print(nulls[nulls > 0.80])

Series([], dtype: float64)


In [9]:
# Se convierten en booleanas las siguientes columnas:

valores_terraza = {'19 metros', 'No acristalada',
       'Gran mirador para 2 sillas y una mesita.', '24m²',
       'Con vistas al retiro', '2 terrazas', '10', 'mtrs=25',
       'Dos terrazas de 60 m² cada uno', '3 terrazas', '80 metros',
       'Pequeña pero muy mona', 'mtrs=0', '20 m²',
       'En una de las habitaciones', '30 m²', 'De unos 8m²',
       'De unos 15m²', '3m²', 'De 30 m². con toldo y luces',
       'En azotea con vistas de madrid', 'En azotea', 'Mas de 18m²',
       'Terraza-patio ajardinado de uso privativo', 'De unos 10m²',
       '15 m²', '6m²', 'De 40 m².', '30m²',
       '2 balcones + una terraza interior', 'En el dormitorio', '5',
       '3 m²', 'Balcones', '100', '6', 'Y un mirador', 'mtrs=8', 'mtrs=5',
       '20', 'Acristalada', '4', '12', '8', '13', '3', '15', '60', '7',
       '15.00 m²', 'Acristalda', 'Con espacio para mesa y tumbonas',
       'Una a pie de salón y otra amplia en planta alta.', 'Solarium',
       'Orientada al sur'}
df['terraza'] = df['terraza'].isin(valores_terraza)

valores_piscina = {'Propia', 'Comunitaria', 'Privada'}
df['piscina'] = df['piscina'].isin(valores_piscina)

valores_trastero ={'Opcional', 'Dentro de la casa', '8m². sótano-1 puerta 10'}
df['trastero'] = df['trastero'].isin(valores_trastero)

valores_calefacción = {'Eléctrica', 'Individual eléctrica', 'Central',
       'Gas natural', 'Individual', 'Aerotermia',
       'Contadores individuales', 'Gasoil', 'Bomba de calor',
       'Incluida en la renta', 'Con contadores individuales',
       'Suelo radiante', 'Bomba frio calor', 'Con contador individual',
       'Central con contador individual', 'Bomba friol calor',
       'Gasoil con contador individual'}
df['calefacción'] = df['calefacción'].isin(valores_calefacción)

valores_ascensor = {'3', '2', 'Precioso de madera', 'Dos ascensores',
       '3 ascensores', 'En la entre-planta.'}
df['ascensor'] = df['ascensor'].isin(valores_ascensor)

valores_aire_acondicionado = {'Frío y calor', 'Frío', 'Individual',
       'Splits y unidades', 'A/a f/c', 'Comunitario', 'Aerotermia',
       'Frío-calor', 'Con bomba de calor', 'Centralizado',
       'A/a por conductos con sistema airzone', 'Tambien suelo radiante',
       'A/a f/c centralizado', 'En toda la casa', 'De ventana',
       'Por conductos independientes', 'Por conductos'}
df['aire acondicionado'] = df['aire acondicionado'].isin(valores_aire_acondicionado)

valores_exterior = {'Exterior calle',
       'Preciosa vistas a patio de manzana.', 'Exterior',
       'A piscina y calle', 'Ladrillo', 'Con vistas al retiro',
       'Con un balcón', 'Exterior patio manzana'}
df['exterior'] = df['exterior'].isin(valores_exterior)

valores_garaje ={'1', '2', 'Más de 2'}
df['garaje'] = df['garaje'].isin(valores_garaje)

valores_jardín = {'Privado', 'Comunitario'}
df['jardín'] = df['jardín'].isin(valores_jardín)

valores_sistema_seguridad = {'con cámaras de seguridad', 'Portero físico',
       'Cámara en todo el edificio', '12 horas/día los 7 días/semana.',
       'vigilancia 24h. con cámaras de seguridad', 'Videovigilancia',
       'Portero', 'Conserje. Doble portal', 'Conserje',
       'Portero residente', '24 horas', 'Portería incluso domingos',
       'Conserje residente', 'Vigilancia 24 horas', '24hs.'}
df['sistema de seguridad'] = df['sistema de seguridad'].isin(valores_sistema_seguridad)

valores_vidrios_dobles = {'Esta muy aislado', 'Rotura puente térmico',
       'Climalit con rotura puente térmico', 'Climalit'}
df['vidrios dobles'] = df['vidrios dobles'].isin(valores_vidrios_dobles)

C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\4133289599.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['terraza'] = df['terraza'].isin(valores_terraza)
C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\4133289599.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['piscina'] = df['piscina'].isin(valores_piscina)
C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\4133289599.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [10]:
# Categorización amueblado

valores_si = {
    'Completamente amueblado', 'Muebles de diseño', 'De forma minimalista y funcional', 
    'Totalmente', 'Con muebles de diseño', 'Cocina amueblada', 'Completo', 'Moderno y actual'
}

valores_semi = {
    'Solo la cocina y los baños', 'Semi-amueblado', 'Semi amueblado',
    'Opcional amueblado/vacio', 'Puede ser sin amueblar y amueblado',
    'Pueden amueblar la 3ra habitación tambine', 'No muy bien. mala calidad y poco ergonómico.'
}

def categorizar_amueblado(valor):
    if pd.isna(valor):
        return np.nan
    if valor in valores_si:
        return 'Si'
    elif valor in valores_semi:
        return 'Semi'
    elif valor is False or valor == 'False':
        return 'No'
    else:
        return 'Si'  

df['amueblado'] = df['amueblado'].apply(categorizar_amueblado)


C:\Users\Marta\AppData\Local\Temp\ipykernel_13792\3819075104.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amueblado'] = df['amueblado'].apply(categorizar_amueblado)


In [11]:
# Mostrar valores nulos ordenados (descendente)
df.isnull().sum().sort_values(ascending=False)


antigüedad                                    1194
barrio                                          30
distrito                                        30
superficie construida                            7
superficie útil                                  5
habitaciones                                     2
garaje                                           0
jardín                                           0
piscina                                          0
planta                                           0
rent_eur_per_month                               0
portero automático                               0
exterior                                         0
referencia                                       0
sistema de seguridad                             0
terraza                                          0
trastero                                         0
puerta blindada                                  0
cocina equipada                                  0
conservación                   

In [12]:
df.to_csv("madrid_rental_properties_cleaned.csv", index=False)

In [ ]:
##############################################################################################################################